In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
from matplotlib import rcParams, cycler
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
start_cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',).load()
start_cnt = start_cnt.drop_channels(['STI 014'])
def car(a):
    return a - np.mean(a, keepdims=True, axis=0)

start_cnt = mne_apply(
    car, start_cnt)

start_cnt = start_cnt.reorder_channels(['C3', 'C4'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

log.info("Resampling train...")
cnt = resample_cnt(start_cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 32.0)
cnt = resample_cnt(cnt, 64.0)
#cnt = mne_apply(
#    lambda a: bandpass_cnt(a, 0, 2, cnt.info['sfreq'],
#                           filt_order=10,
#                           axis=1), cnt)

train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)
cnt_bandpassed =  mne_apply(
    lambda a: bandpass_cnt(a, 8, 13, cnt.info['sfreq'],
                           filt_order=10,
                           axis=1), cnt)
alpha_set = create_signal_target_from_raw_mne(cnt_bandpassed, marker_def, ival)
x_alpha_right = alpha_set.X[alpha_set.y == 0]

x_alpha_rest = alpha_set.X[alpha_set.y == 2]

alpha_a = np_to_var(x_alpha_right[:160,0:1,:,None], dtype=np.float32).cuda()

alpha_b = np_to_var(x_alpha_rest[:160,0:1,:,None], dtype=np.float32).cuda()
inputs_alpha = [alpha_a, alpha_b]


In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True)),
         
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True))
    )

In [ ]:
from reversible.util import set_random_seeds
set_random_seeds(2019011641, True)
feature_model = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 2 x 32
    rev_block(2,32),
    rev_block(2,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 4 x 16
    rev_block(4,32),
    rev_block(4,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 8 x 8
    rev_block(8,32),
    rev_block(8,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 16 x 4
    rev_block(16,32),
    rev_block(16,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 32 x 2
    rev_block(32,32),
    rev_block(32,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 64 x 1
    rev_block(64,64),
    rev_block(64,64),
    ViewAs((-1,64,1,1), (-1,64)),
)

feature_model.cuda()
n_clusters = len(inputs_alpha)
n_dims = inputs_alpha[0].shape[2]

coefs = th.zeros(len(inputs_alpha[0]), 2, device='cuda', requires_grad=True)


from copy import deepcopy
optimizer = th.optim.Adam(
                          [
    {'params': list(feature_model.parameters()),
    'lr': 1e-3,
    'weight_decay': 0},], betas=(0,0.9))

optim_dist = th.optim.Adam(
                          [
    {'params': [coefs],
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

In [ ]:
n_epochs = 1001
for i_epoch in range(n_epochs):
    this_ins = inputs_alpha[0]
    all_coefs = th.cat((th.zeros(len(this_ins), 16, device='cuda'),
                       coefs[:,0:1], # 16
                       th.zeros(len(this_ins), 31, device='cuda'),
                       coefs[:,1:2],
                       th.zeros(len(this_ins), 15, device='cuda'),), dim=1)

    inverted = invert(feature_model, all_coefs)


    loss = th.mean(th.sqrt(th.sum((inverted.squeeze() - this_ins.squeeze()) ** 2, dim=1)))
    optimizer.zero_grad()
    optim_dist.zero_grad()
    loss.backward()
    optimizer.step()
    optim_dist.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
        print("Loss: {:.2f}".format(loss.item()))

        fig = plt.figure(figsize=(4,4))
        plt.plot(var_to_np(coefs)[:,0],var_to_np(coefs)[:,1], ls='', marker='o')
        plt.title("Coefs")
        display(fig)
        plt.close(fig)
        inverted = invert(feature_model, all_coefs)
        fig, axes = plt.subplots(3,8, figsize=(18,5), sharex=True, sharey=True)
        for i_in, ax in enumerate(axes.flatten()):
            ax.plot(var_to_np(inverted[i_in].squeeze()), color=seaborn.color_palette()[0])
            ax.plot(var_to_np(this_ins[i_in].squeeze()), color='black')
        display(fig)
        plt.close(fig)
        
        mean = th.mean(all_coefs.detach(), dim=0)

        m1, s1 = th.mean(all_coefs[:,16]), th.std(all_coefs[:,16])
        m2, s2 = th.mean(all_coefs[:,48]), th.std(all_coefs[:,48])

        n_vals = 9
        vals_1 = th.linspace((m1 - s1).item(), (m1 + s1).item(), n_vals)
        vals_2 = th.linspace((m2 - s2).item(), (m2 + s2).item(), n_vals)


        grid = np.zeros((n_vals, n_vals, 64))

        for i_val_1, val_1 in enumerate(vals_1):
            for i_val_2, val_2 in enumerate(vals_2):
                this = mean.clone()
                this.data[16] = val_1
                this.data[48] = val_2
                inverted= invert(feature_model, this.unsqueeze(0))
                grid[i_val_1, i_val_2] = var_to_np(inverted).squeeze()

        fig, axes = plt.subplots(n_vals, n_vals, figsize=(14,10), sharex=True, sharey=True)
        for i_val_1, i_val_2 in itertools.product(range(n_vals), range(n_vals)):
            # note axes are first rows, then columns (i.e., first y then x)
            # and starts at top so at high y
            axes[grid.shape[1] - i_val_2 - 1, i_val_1].plot(grid[i_val_1, i_val_2])
        display(fig)
        plt.close(fig)
        
        fig = plt.figure(figsize=(8,2))
        plt.plot(np.mean(np.abs(np.fft.rfft(grid)), axis=(0,1)))
        plt.title("Spectrum")
        display(fig)
        plt.close(fig)
        
        grid_phases = np.angle(np.mean(np.fft.rfft(grid), axis=(2)))
        fig = plt.figure(figsize=(8,8))
        plt.imshow(grid_phases, vmin=-np.pi, vmax=np.pi, cmap=cm.twilight)
        cbar = plt.colorbar()
        cbar.set_label("Phase [Radians]")
        display(fig)
        plt.close(fig)
        
        grid_amps = np.abs(np.mean(np.fft.rfft(grid), axis=(2)))
        fig = plt.figure(figsize=(8,8))
        plt.imshow(grid_amps, vmin=0, cmap=cm.Reds)
        cbar = plt.colorbar()
        cbar.set_label("Amplitude")
        display(fig)
        plt.close(fig)

In [ ]:
vals_1

In [ ]:
vals_2

In [ ]:
print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
print("Loss: {:.2f}".format(loss.item()))

fig = plt.figure(figsize=(8,8))
plt.plot(var_to_np(coefs)[:,0],var_to_np(coefs)[:,1], ls='', marker='o')
plt.plot(np.mean(var_to_np(coefs)[:,0]),np.mean(var_to_np(coefs)[:,1]), ls='', marker='^', markersize=12, color='black')
plt.title("Coefs")
display(fig)
plt.close(fig)

In [ ]:
inverted = invert(feature_model, all_coefs)
fig, axes = plt.subplots(6,8, figsize=(18,12), sharex=True, sharey=True)
for i_in, ax in enumerate(axes.flatten()):
    ax.plot(var_to_np(inverted[i_in].squeeze()), color=seaborn.color_palette()[0])
    ax.plot(var_to_np(this_ins[i_in].squeeze()), color='black')
display(fig)
plt.close(fig)
mean = th.mean(all_coefs.detach(), dim=0)

m1, s1 = th.mean(all_coefs[:,16]), th.std(all_coefs[:,16])
m2, s2 = th.mean(all_coefs[:,48]), th.std(all_coefs[:,48])

n_vals = 9
vals_1 = th.linspace((m1 - s1).item(), (m1 + s1).item(), n_vals)
vals_2 = th.linspace((m2 - s2).item(), (m2 + s2).item(), n_vals)


grid = np.zeros((n_vals, n_vals, 64))

for i_val_1, val_1 in enumerate(vals_1):
    for i_val_2, val_2 in enumerate(vals_2):
        this = mean.clone()
        this.data[16] = val_1
        this.data[48] = val_2
        inverted= invert(feature_model, this.unsqueeze(0))
        grid[i_val_1, i_val_2] = var_to_np(inverted).squeeze()
        
fig, axes = plt.subplots(n_vals, n_vals, figsize=(14,10), sharex=True, sharey=True)
for i_val_1, i_val_2 in itertools.product(range(n_vals), range(n_vals)):
    # note axes are first rows, then columns (i.e., first y then x)
    # and starts at top so at high y
    axes[grid.shape[1] - i_val_2 - 1, i_val_1].plot(grid[i_val_1, i_val_2])
display(fig)
plt.close(fig)

In [ ]:
all_outs = feature_model(this_ins)
fig = plt.figure(figsize=(8,8))
plt.plot(var_to_np(coefs)[:,0],var_to_np(coefs)[:,1], ls='', marker='o',
        label='Optimized coeffs', alpha=0.7)
plt.plot(np.mean(var_to_np(coefs)[:,0]),np.mean(var_to_np(coefs)[:,1]),
         ls='', marker='^', markersize=12, color=seaborn.color_palette()[2], alpha=0.7)
plt.plot(var_to_np(all_outs)[:,16],var_to_np(all_outs)[:,48], ls='', marker='o', alpha=0.7,
        label="Encodings")
plt.plot(np.mean(var_to_np(all_outs)[:,16]),np.mean(var_to_np(all_outs)[:,48]),
         ls='', marker='^', markersize=12, color=seaborn.color_palette()[3], alpha=0.7)
plt.title("Coefs")
plt.legend()
display(fig)
plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(8,2))
plt.plot(np.mean(np.abs(np.fft.rfft(grid)), axis=(0,1)))
plt.title("Spectrum")
display(fig)
plt.close(fig)

In [ ]:
grid_phases = np.angle(np.mean(np.fft.rfft(grid), axis=(2)))
plt.figure(figsize=(8,8))
plt.imshow(grid_phases, vmin=-np.pi, vmax=np.pi, cmap=cm.twilight)
cbar = plt.colorbar()
cbar.set_label("Phase [Radians]")

In [ ]:
grid_amps = np.abs(np.mean(np.fft.rfft(grid), axis=(2)))

plt.figure(figsize=(8,8))
plt.imshow(grid_amps, vmin=0, cmap=cm.Reds)
cbar = plt.colorbar()
cbar.set_label("Amplitude")

In [ ]:
grid_amps = np.mean(grid * grid, axis=(2))

plt.figure(figsize=(8,8))
plt.imshow(grid_amps, vmin=0, cmap=cm.Reds)
cbar = plt.colorbar()
cbar.set_label("Energy")

In [ ]:
grid_phases = np.angle(np.mean(np.fft.rfft(grid), axis=(2)))
plt.figure(figsize=(8,8))
plt.imshow(grid_phases, vmin=-np.pi, vmax=np.pi, cmap=cm.twilight)
cbar = plt.colorbar()
cbar.set_label("Phase [Radians]")

In [ ]:
grid_amps = np.abs(np.mean(np.fft.rfft(grid), axis=(2)))

plt.figure(figsize=(8,8))
plt.imshow(grid_amps, vmin=0, cmap=cm.Reds)
cbar = plt.colorbar()
cbar.set_label("Amplitude")

In [ ]:
grid.shape

In [ ]:
grid_amps = np.mean(grid * grid, axis=(2))

plt.figure(figsize=(8,8))
plt.imshow(grid_amps, vmin=0, cmap=cm.Reds)
cbar = plt.colorbar()
cbar.set_label("Energy")

In [ ]:
plt.plot(grid[0,-1])
plt.plot(grid[-1,-1])